In [37]:
import re
import os
from pathlib import Path

import pandas as pd
import numpy as np
import tifffile
from tifffile import TiffFile
from ome_types import from_tiff, from_xml, to_xml, model
from ome_types.model.simple_types import UnitsLength


from multiplex_imaging_pipeline.utils import listfiles

In [4]:
output_dir = '/diskmnt/Projects/Users/estorrs/imaging-analysis/data/brca_subtype/htan_upload'
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [53]:
metadata = pd.read_csv('/diskmnt/Projects/Users/estorrs/sandbox/codex_specimens.tsv', sep='\t')
metadata['id'] = [f'{sid}_{date}' for sid, date in zip(metadata['specimen'], metadata['date'])]
metadata = metadata.drop_duplicates(subset='id')
metadata.index = metadata['id']
metadata

,run_name,omero_run_name,date,case,specimen,bbox,filepath,id
id,,,,,,,,
C3L-00970-13u2_20231220,20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00...,2023-12-20_C3L-00970__C3N-00733__PA00002352,20231220,C3L-00970,C3L-00970-13u2,"0,15000,6000,28000",/diskmnt/primary/CODEX/HTAN/20231220_Human_CCR...,C3L-00970-13u2_20231220
C3N-00733-12U2_20231220,20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00...,2023-12-20_C3L-00970__C3N-00733__PA00002352,20231220,C3N-00733,C3N-00733-12U2,"18000,42000,6000,23000",/diskmnt/primary/CODEX/HTAN/20231220_Human_CCR...,C3N-00733-12U2_20231220
PA00002352U1_20231220,20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00...,2023-12-20_C3L-00970__C3N-00733__PA00002352,20231220,PA00002352,PA00002352U1,"45000,66960,0,28800",/diskmnt/primary/CODEX/HTAN/20231220_Human_CCR...,PA00002352U1_20231220
C3L-01287-11Us1_20231219,20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N...,2023-12-19_C3L-01287__C3N-01200,20231219,C3L-01287,C3L-01287-11Us1,"0,18000,4000,29760",/diskmnt/primary/CODEX/HTAN/20231219_Human_CCR...,C3L-01287-11Us1_20231219
C3N-01200-11Us1_2_20231219,20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N...,2023-12-19_C3L-01287__C3N-01200,20231219,C3N-01200,C3N-01200-11Us1_2,"33000,57000,6000,23000",/diskmnt/primary/CODEX/HTAN/20231219_Human_CCR...,C3N-01200-11Us1_2_20231219
...,...,...,...,...,...,...,...,...
Human_Kidney_292_20210928,20210928-Human Kidney 292,2021-09-28_Human_Kidney_292,20210928,Human Kidney 292,Human_Kidney_292,NaN,/diskmnt/primary/CODEX/HTAN/20210928-Human_Kid...,Human_Kidney_292_20210928
HT264P1-S1H2A2_20210720,20210720-HT264P1-S1H2A2,2021-07-20_HT264P1,20210720,HT264P1,HT264P1-S1H2A2,NaN,/diskmnt/primary/CODEX/HTAN/20210720-HT264P1-S...,HT264P1-S1H2A2_20210720
HT339B1-S1H3A1_20211019,20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,20211019,HT339B1,HT339B1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,HT339B1-S1H3A1_20211019


In [54]:
# from collections import Counter
# Counter(metadata.index).most_common()

In [55]:
inclusion = pd.read_csv(
    '/diskmnt/Projects/Users/estorrs/imaging-analysis/data/brca_subtype/htan_upload/brca_subtype_inclusion.txt',
    sep='\t'
)
inclusion['id'] = [f'{sid}_{date}' for sid, date in zip(inclusion['specimen'], inclusion['date'])]
inclusion = inclusion.drop_duplicates(subset='id')
inclusion.index = inclusion['id']
inclusion

,run_name,omero_run_name,date,case,specimen,included,id
id,,,,,,,
C3L-00970-13u2_20231220,20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00...,2023-12-20_C3L-00970__C3N-00733__PA00002352,20231220,C3L-00970,C3L-00970-13u2,no,C3L-00970-13u2_20231220
C3N-00733-12U2_20231220,20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00...,2023-12-20_C3L-00970__C3N-00733__PA00002352,20231220,C3N-00733,C3N-00733-12U2,no,C3N-00733-12U2_20231220
PA00002352U1_20231220,20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00...,2023-12-20_C3L-00970__C3N-00733__PA00002352,20231220,PA00002352,PA00002352U1,no,PA00002352U1_20231220
C3L-01287-11Us1_20231219,20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N...,2023-12-19_C3L-01287__C3N-01200,20231219,C3L-01287,C3L-01287-11Us1,no,C3L-01287-11Us1_20231219
C3N-01200-11Us1_2_20231219,20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N...,2023-12-19_C3L-01287__C3N-01200,20231219,C3N-01200,C3N-01200-11Us1_2,no,C3N-01200-11Us1_2_20231219
...,...,...,...,...,...,...,...
Human_Kidney_292_20210928,20210928-Human Kidney 292,2021-09-28_Human_Kidney_292,20210928,Human Kidney 292,Human_Kidney_292,no,Human_Kidney_292_20210928
HT264P1-S1H2A2_20210720,20210720-HT264P1-S1H2A2,2021-07-20_HT264P1,20210720,HT264P1,HT264P1-S1H2A2,no,HT264P1-S1H2A2_20210720
HT339B1-S1H3A1_20211019,20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,20211019,HT339B1,HT339B1-S1H3A1,no,HT339B1-S1H3A1_20211019


In [56]:
metadata['included'] = inclusion.loc[metadata.index.to_list(), 'included']
metadata = metadata[metadata['included']=='yes']
metadata = metadata.sort_index()
metadata

,run_name,omero_run_name,date,case,specimen,bbox,filepath,id,included
id,,,,,,,,,
HT110B1-S1H4_20221216,20221216_BRCA_,2022-12-16_HT110B1__HT243B1__HT305B1,20221216,HT110B1,HT110B1-S1H4,"33000,48000,9000,25000",/diskmnt/primary/CODEX/HTAN/HT243B1-S1H4_HT110...,HT110B1-S1H4_20221216,yes
HT171B1-S1H9A1-4_left_20230512,HT171B1-S1H9A1-4_TOP_HT243B1-S1H4A4_HT271B1-S1...,2023-05-12_HT171B1__HT243B1__HT271B1__HT308B1,20230512,HT171B1,HT171B1-S1H9A1-4_left,"0,9000,2000,15000",/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...,HT171B1-S1H9A1-4_left_20230512,yes
HT171B1-S1H9A1-4_right_20230512,HT171B1-S1H9A1-4_TOP_HT243B1-S1H4A4_HT271B1-S1...,2023-05-12_HT171B1__HT243B1__HT271B1__HT308B1,20230512,HT171B1,HT171B1-S1H9A1-4_right,"0,9000,17000,31000",/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...,HT171B1-S1H9A1-4_right_20230512,yes
HT171B1-S1H9A1-A4_left_20230526,052623_HT171B1-S1H9A1-A4_top_HT243B1-S1H4A4_HT...,2023-05-26_HT171B1__HT243B1__HT271B1__HT308B1,20230526,HT171B1,HT171B1-S1H9A1-A4_left,"0,7000,0,11000",/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H...,HT171B1-S1H9A1-A4_left_20230526,yes
HT171B1-S1H9A1-A4_right_20230526,052623_HT171B1-S1H9A1-A4_top_HT243B1-S1H4A4_HT...,2023-05-26_HT171B1__HT243B1__HT271B1__HT308B1,20230526,HT171B1,HT171B1-S1H9A1-A4_right,"1000,10000,16000,28000",/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H...,HT171B1-S1H9A1-A4_right_20230526,yes
HT206B1-H1_20220625,20220625-HT206B1-H1,2022-06-25_HT206B1,20220625,HT206B1,HT206B1-H1,NaN,/diskmnt/primary/CODEX/HTAN/20220625-HT206B1-H...,HT206B1-H1_20220625,yes
HT206B1-S1H2L4_20221028,20221028_HT206B1-S1H2L4_LRG_HT427P1_S1H1A3_SML,2022-10-28_HT206B1__HT427P1,20221028,HT206B1,HT206B1-S1H2L4,"1000,13000,3000,14000",/diskmnt/primary/CODEX/HTAN/HT206B1-S1H2L4_LRG...,HT206B1-S1H2L4_20221028,yes
HT206B1-S1H5_left_20230701,07012023_BRCA_HT206B1-S1H5_TOP_HT297B1-S1H1_HT...,2023-07-01_HT206B1__HT297B1__HT339B2__HT565B1,20230701,HT206B1,HT206B1-S1H5_left,"0,10000,2000,13000",/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...,HT206B1-S1H5_left_20230701,yes
HT206B1-S1H5_right_20230701,07012023_BRCA_HT206B1-S1H5_TOP_HT297B1-S1H1_HT...,2023-07-01_HT206B1__HT297B1__HT339B2__HT565B1,20230701,HT206B1,HT206B1-S1H5_right,"0,10000,17000,26000",/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...,HT206B1-S1H5_right_20230701,yes


In [57]:
metadata.shape

(49, 9)

#### generate level 2 image metadata file

In [58]:
sample_fp_tups = [(sid, fp) for sid, fp in zip(metadata.index.to_list(), metadata['filepath'])]

In [59]:
assay_type = 'CODEX'

In [72]:
df = pd.DataFrame.from_dict({
    'Component': ['ImagingLevel2'] * len(sample_fp_tups),
    'File Format': ['OME-TIFF'] * len(sample_fp_tups),
    'HTAN Participant ID': [''] * len(sample_fp_tups),
    'HTAN Parent Biospecimen ID': [''] * len(sample_fp_tups),
    'HTAN Data File ID': [''] * len(sample_fp_tups),
#     'HTAN Parent Data File ID': ['matt provides'] * len(sample_fp_tups),
    'Imaging Assay Type': [assay_type] * len(sample_fp_tups),
    'Protocol Link': [''] * len(sample_fp_tups),
    'Workflow Start Datetime': [''] * len(sample_fp_tups),
    'Workflow End Datetime': [''] * len(sample_fp_tups),
    'Software and Version': [''] * len(sample_fp_tups),
    'Image Acquisition Date': [''] * len(sample_fp_tups),
    'Microscope': [''] * len(sample_fp_tups),
    'Objective': [''] * len(sample_fp_tups),
    'NominalMagnification': [''] * len(sample_fp_tups),
    'LensNA': [''] * len(sample_fp_tups),
    'WorkingDistance': [''] * len(sample_fp_tups),
    'WorkingDistanceUnit': [''] * len(sample_fp_tups),
    'Immersion': [''] * len(sample_fp_tups),
    'Passed QC': [True] * len(sample_fp_tups),
    'Pyramid': [True] * len(sample_fp_tups),
    'Tseries': [False] * len(sample_fp_tups),
    'Zstack': [False] * len(sample_fp_tups),
    'Comment': [''] * len(sample_fp_tups),
    'FOV number': [''] * len(sample_fp_tups),
    'FOVX': [''] * len(sample_fp_tups),
    'FOVXUnit': [''] * len(sample_fp_tups),
    'FOVY': [''] * len(sample_fp_tups),
    'FOVYUnit': [''] * len(sample_fp_tups),
    'Frame Averaging': [''] * len(sample_fp_tups),
    'PhysicalSizeZ': [''] * len(sample_fp_tups),
    'PhysicalSizeZUnit': [''] * len(sample_fp_tups)
})

fns, imid, do, psx, pux, psy, puy, pbe, pc, sc, st, sx, sy, sz, pt = [[] for x in range(15)]
for sample, fp in sample_fp_tups:
    tif = TiffFile(fp)
    ome = from_xml(tif.ome_metadata)
    im = ome.images[0]
    
    fns.append(fp.split('/')[-1])
    imid.append(im.id)
    do.append(im.pixels.dimension_order.value)
    psx.append(im.pixels.physical_size_x)
    pux.append(im.pixels.physical_size_x_unit.value)
    psy.append(im.pixels.physical_size_y)
    puy.append(im.pixels.physical_size_y_unit.value)
    pbe.append(im.pixels.big_endian)
    pc.append(im.pixels.tiff_data_blocks[0].plane_count)
    sc.append(im.pixels.size_c)
    st.append(im.pixels.size_t)
    sx.append(im.pixels.size_x)
    sy.append(im.pixels.size_y)
    sz.append(im.pixels.size_z)
    pt.append(im.pixels.type.value)
    
    
df = pd.merge(df,
             pd.DataFrame.from_dict({
                 'Filename': fns,
                 'Channel Metadata Filename': [f.replace('.ome.tiff', '.csv') for f in fns],
                 'Image ID': imid,
                 'DimensionOrder': do,
                 'PhysicalSizeX': psx,
                 'PhysicalSizeXUnit': pux,
                 'PhysicalSizeY': psy,
                 'PhysicalSizeYUnit': puy,
                 'Pixels BigEndian': pbe,
                 'PlaneCount': pc,
                 'SizeC': sc,
                 'SizeT': st,
                 'SizeX': sx,
                 'SizeY': sy,
                 'SizeZ': sz,
                 'PixelType': pt
             }), left_index=True, right_index=True)

order = ['Component', 'Filename', 'File Format', 'HTAN Participant ID', 'HTAN Parent Biospecimen ID',
         'HTAN Data File ID', 'Channel Metadata Filename', 'Imaging Assay Type', 'Protocol Link',
         'Workflow Start Datetime', 'Workflow End Datetime', 'Software and Version',
         'Image Acquisition Date', 'Microscope', 'Objective', 'NominalMagnification', 'LensNA', 'WorkingDistance',
         'WorkingDistanceUnit', 'Immersion', 'Passed QC', 'Pyramid', 'Tseries', 'Zstack', 'Comment',
         'FOV number', 'FOVX', 'FOVXUnit', 'FOVY', 'FOVYUnit', 'Frame Averaging',
        'Image ID', 'DimensionOrder', 'PhysicalSizeX', 'PhysicalSizeXUnit', 'PhysicalSizeY', 'PhysicalSizeYUnit',
         'PhysicalSizeZ', 'PhysicalSizeZUnit',
        'Pixels BigEndian', 'PlaneCount', 'SizeC', 'SizeT', 'SizeX', 'SizeY', 'SizeZ', 'PixelType']

df = df[order]
df

,Component,Filename,File Format,HTAN Participant ID,HTAN Parent Biospecimen ID,HTAN Data File ID,Channel Metadata Filename,Imaging Assay Type,Protocol Link,Workflow Start Datetime,...,PhysicalSizeZ,PhysicalSizeZUnit,Pixels BigEndian,PlaneCount,SizeC,SizeT,SizeX,SizeY,SizeZ,PixelType
0,ImagingLevel2,HT110B1-S1H4__20221216.ome.tiff,OME-TIFF,,,,HT110B1-S1H4__20221216.csv,CODEX,,,...,,,False,26,26,1,16000,15000,1,uint8
1,ImagingLevel2,HT171B1-S1H9A1-4_left__20230512.ome.tiff,OME-TIFF,,,,HT171B1-S1H9A1-4_left__20230512.csv,CODEX,,,...,,,False,33,33,1,13000,9000,1,uint8
2,ImagingLevel2,HT171B1-S1H9A1-4_right__20230512.ome.tiff,OME-TIFF,,,,HT171B1-S1H9A1-4_right__20230512.csv,CODEX,,,...,,,False,33,33,1,14000,9000,1,uint8
3,ImagingLevel2,HT171B1-S1H9A1-A4_left__20230526.ome.tiff,OME-TIFF,,,,HT171B1-S1H9A1-A4_left__20230526.csv,CODEX,,,...,,,False,34,34,1,11000,7000,1,uint8
4,ImagingLevel2,HT171B1-S1H9A1-A4_right__20230526.ome.tiff,OME-TIFF,,,,HT171B1-S1H9A1-A4_right__20230526.csv,CODEX,,,...,,,False,34,34,1,12000,9000,1,uint8
5,ImagingLevel2,HT206B1-H1__20220625.ome.tiff,OME-TIFF,,,,HT206B1-H1__20220625.csv,CODEX,,,...,,,False,29,29,1,14784,20160,1,uint8
6,ImagingLevel2,HT206B1-S1H2L4__20221028.ome.tiff,OME-TIFF,,,,HT206B1-S1H2L4__20221028.csv,CODEX,,,...,,,False,20,20,1,11000,12000,1,uint8
7,ImagingLevel2,HT206B1-S1H5_left__20230701.ome.tiff,OME-TIFF,,,,HT206B1-S1H5_left__20230701.csv,CODEX,,,...,,,False,37,37,1,11000,10000,1,uint8
8,ImagingLevel2,HT206B1-S1H5_right__20230701.ome.tiff,OME-TIFF,,,,HT206B1-S1H5_right__20230701.csv,CODEX,,,...,,,False,37,37,1,9000,10000,1,uint8
9,ImagingLevel2,HT243B1-S1H4A4__20230420.ome.tiff,OME-TIFF,,,,HT243B1-S1H4A4__20230420.csv,CODEX,,,...,,,False,31,31,1,16000,10000,1,uint8


In [77]:
df.to_csv(os.path.join(output_dir, 'file_level_metadata.csv'), sep=',', index=False, encoding='UTF-8')

#### channel metadata for each file

In [66]:
for sample, fp in sample_fp_tups:
    channel_id, channel_name = [], []
    print(sample, fp)
    tif = TiffFile(fp)
    m = re.sub(r'<Acquis.*AcquisitionDate>', r'', tif.ome_metadata)
    ome = from_xml(m)
    im = ome.images[0]
    for channel in im.pixels.channels:
        channel_id.append(channel.id)
        channel_name.append(channel.name)
        
    df = pd.DataFrame.from_dict({
        'Channel ID': channel_id,
        'Channel Name': channel_name,
        'Passed QC': [True] * len(channel_name)
    })
    df.to_csv(os.path.join(output_dir, f'{sample}.csv'), sep=',', index=False)

HT110B1-S1H4_20221216 /diskmnt/primary/CODEX/HTAN/HT243B1-S1H4_HT110B1-S1H4_HT305B1-S1H1/HT110B1-S1H4__20221216.ome.tiff
HT171B1-S1H9A1-4_left_20230512 /diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B1-S1H9A1-4_TOP_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4/HT171B1-S1H9A1-4_left__20230512.ome.tiff
HT171B1-S1H9A1-4_right_20230512 /diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B1-S1H9A1-4_TOP_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4/HT171B1-S1H9A1-4_right__20230512.ome.tiff
HT171B1-S1H9A1-A4_left_20230526 /diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H9A1-A4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/HT171B1-S1H9A1-A4_left__20230526.ome.tiff
HT171B1-S1H9A1-A4_right_20230526 /diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H9A1-A4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/HT171B1-S1H9A1-A4_right__20230526.ome.tiff
HT206B1-H1_20220625 /diskmnt/primary/CODEX/HTAN/20220625-HT206B1-H1/HT206B1-H1__20220625.ome.tiff
HT206B1-S1H2L4_20221028 /diskmnt/primary/CODEX/HTAN/HT206B1-S

#### table for matt so he can link files to table

In [87]:
link = metadata[['case', 'specimen', 'date', 'filepath']].copy()
link

,case,specimen,date,filepath
id,,,,
HT110B1-S1H4_20221216,HT110B1,HT110B1-S1H4,20221216,/diskmnt/primary/CODEX/HTAN/HT243B1-S1H4_HT110...
HT171B1-S1H9A1-4_left_20230512,HT171B1,HT171B1-S1H9A1-4_left,20230512,/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...
HT171B1-S1H9A1-4_right_20230512,HT171B1,HT171B1-S1H9A1-4_right,20230512,/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...
HT171B1-S1H9A1-A4_left_20230526,HT171B1,HT171B1-S1H9A1-A4_left,20230526,/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H...
HT171B1-S1H9A1-A4_right_20230526,HT171B1,HT171B1-S1H9A1-A4_right,20230526,/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H...
HT206B1-H1_20220625,HT206B1,HT206B1-H1,20220625,/diskmnt/primary/CODEX/HTAN/20220625-HT206B1-H...
HT206B1-S1H2L4_20221028,HT206B1,HT206B1-S1H2L4,20221028,/diskmnt/primary/CODEX/HTAN/HT206B1-S1H2L4_LRG...
HT206B1-S1H5_left_20230701,HT206B1,HT206B1-S1H5_left,20230701,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...
HT206B1-S1H5_right_20230701,HT206B1,HT206B1-S1H5_right,20230701,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...


In [88]:
link['Filename'] = [x.split('/')[-1] for x in link['filepath']]
link['katmai Channel Metadata filepath'] = [os.path.join(output_dir, f'{x}.csv') for x in link.index.to_list()]
link['Channel Metadata Filename'] = [f'{x}.csv' for x in link.index.to_list()]
link.columns = [c if c!= 'filepath' else 'katmai filepath' for c in link.columns]
link

,case,specimen,date,katmai filepath,Filename,katmai Channel Metadata filepath,Channel Metadata Filename
id,,,,,,,
HT110B1-S1H4_20221216,HT110B1,HT110B1-S1H4,20221216,/diskmnt/primary/CODEX/HTAN/HT243B1-S1H4_HT110...,HT110B1-S1H4__20221216.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT110B1-S1H4_20221216.csv
HT171B1-S1H9A1-4_left_20230512,HT171B1,HT171B1-S1H9A1-4_left,20230512,/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...,HT171B1-S1H9A1-4_left__20230512.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT171B1-S1H9A1-4_left_20230512.csv
HT171B1-S1H9A1-4_right_20230512,HT171B1,HT171B1-S1H9A1-4_right,20230512,/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...,HT171B1-S1H9A1-4_right__20230512.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT171B1-S1H9A1-4_right_20230512.csv
HT171B1-S1H9A1-A4_left_20230526,HT171B1,HT171B1-S1H9A1-A4_left,20230526,/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H...,HT171B1-S1H9A1-A4_left__20230526.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT171B1-S1H9A1-A4_left_20230526.csv
HT171B1-S1H9A1-A4_right_20230526,HT171B1,HT171B1-S1H9A1-A4_right,20230526,/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H...,HT171B1-S1H9A1-A4_right__20230526.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT171B1-S1H9A1-A4_right_20230526.csv
HT206B1-H1_20220625,HT206B1,HT206B1-H1,20220625,/diskmnt/primary/CODEX/HTAN/20220625-HT206B1-H...,HT206B1-H1__20220625.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT206B1-H1_20220625.csv
HT206B1-S1H2L4_20221028,HT206B1,HT206B1-S1H2L4,20221028,/diskmnt/primary/CODEX/HTAN/HT206B1-S1H2L4_LRG...,HT206B1-S1H2L4__20221028.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT206B1-S1H2L4_20221028.csv
HT206B1-S1H5_left_20230701,HT206B1,HT206B1-S1H5_left,20230701,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...,HT206B1-S1H5_left__20230701.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT206B1-S1H5_left_20230701.csv
HT206B1-S1H5_right_20230701,HT206B1,HT206B1-S1H5_right,20230701,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...,HT206B1-S1H5_right__20230701.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT206B1-S1H5_right_20230701.csv


In [94]:
import hashlib
def get_md5sum(fp):
    with open(fp, 'rb') as f:
        data = f.read()    
        return hashlib.md5(data).hexdigest()

In [96]:
link['md5'] = [get_md5sum(fp) for fp in link['katmai filepath']]
link['md5 channel metadata file'] = [get_md5sum(fp) for fp in link['katmai Channel Metadata filepath']]
link

,case,specimen,date,katmai filepath,Filename,katmai Channel Metadata filepath,Channel Metadata Filename,md5,md5 channel metadata file
id,,,,,,,,,
HT110B1-S1H4_20221216,HT110B1,HT110B1-S1H4,20221216,/diskmnt/primary/CODEX/HTAN/HT243B1-S1H4_HT110...,HT110B1-S1H4__20221216.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT110B1-S1H4_20221216.csv,b880f7f6a29ae3baeca397abd47a9dd8,10247040a2ab142c6ce1910225a29022
HT171B1-S1H9A1-4_left_20230512,HT171B1,HT171B1-S1H9A1-4_left,20230512,/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...,HT171B1-S1H9A1-4_left__20230512.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT171B1-S1H9A1-4_left_20230512.csv,8cba6e147fcffb112a0490d138fe1db5,8cab6bfb5753663c397d2ac196971494
HT171B1-S1H9A1-4_right_20230512,HT171B1,HT171B1-S1H9A1-4_right,20230512,/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...,HT171B1-S1H9A1-4_right__20230512.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT171B1-S1H9A1-4_right_20230512.csv,9f9773533c6111e9cb030dbdb3260582,8cab6bfb5753663c397d2ac196971494
HT171B1-S1H9A1-A4_left_20230526,HT171B1,HT171B1-S1H9A1-A4_left,20230526,/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H...,HT171B1-S1H9A1-A4_left__20230526.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT171B1-S1H9A1-A4_left_20230526.csv,e6fffc8c86acf6f0046cf25274eb2cdb,152c1b2b48a7e0c97de235b6b0d8d981
HT171B1-S1H9A1-A4_right_20230526,HT171B1,HT171B1-S1H9A1-A4_right,20230526,/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H...,HT171B1-S1H9A1-A4_right__20230526.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT171B1-S1H9A1-A4_right_20230526.csv,dffdc0264ca6a57f952797f009b0256c,152c1b2b48a7e0c97de235b6b0d8d981
HT206B1-H1_20220625,HT206B1,HT206B1-H1,20220625,/diskmnt/primary/CODEX/HTAN/20220625-HT206B1-H...,HT206B1-H1__20220625.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT206B1-H1_20220625.csv,379826a5ffbbc6d03ca34a54015cc8cd,9b0e6fa14075adab187fb199a5743f66
HT206B1-S1H2L4_20221028,HT206B1,HT206B1-S1H2L4,20221028,/diskmnt/primary/CODEX/HTAN/HT206B1-S1H2L4_LRG...,HT206B1-S1H2L4__20221028.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT206B1-S1H2L4_20221028.csv,471df04bd3b1d2c9e6fcf977319ce83a,170eb95c2b9e63b159f16ed03815de03
HT206B1-S1H5_left_20230701,HT206B1,HT206B1-S1H5_left,20230701,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...,HT206B1-S1H5_left__20230701.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT206B1-S1H5_left_20230701.csv,32536b3b808f836f4c062d29b952bb25,bd940309f07f1859404c3de08fc604eb
HT206B1-S1H5_right_20230701,HT206B1,HT206B1-S1H5_right,20230701,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...,HT206B1-S1H5_right__20230701.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,HT206B1-S1H5_right_20230701.csv,f7e4b6328b5032475eadb5bcb543e4d9,bd940309f07f1859404c3de08fc604eb


In [98]:
link = link[['case', 'specimen', 'date', 'katmai filepath', 'md5',
       'Filename',
       'katmai Channel Metadata filepath', 'md5 channel metadata file', 'Channel Metadata Filename']]
link

,case,specimen,date,katmai filepath,md5,Filename,katmai Channel Metadata filepath,md5 channel metadata file,Channel Metadata Filename
id,,,,,,,,,
HT110B1-S1H4_20221216,HT110B1,HT110B1-S1H4,20221216,/diskmnt/primary/CODEX/HTAN/HT243B1-S1H4_HT110...,b880f7f6a29ae3baeca397abd47a9dd8,HT110B1-S1H4__20221216.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,10247040a2ab142c6ce1910225a29022,HT110B1-S1H4_20221216.csv
HT171B1-S1H9A1-4_left_20230512,HT171B1,HT171B1-S1H9A1-4_left,20230512,/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...,8cba6e147fcffb112a0490d138fe1db5,HT171B1-S1H9A1-4_left__20230512.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,8cab6bfb5753663c397d2ac196971494,HT171B1-S1H9A1-4_left_20230512.csv
HT171B1-S1H9A1-4_right_20230512,HT171B1,HT171B1-S1H9A1-4_right,20230512,/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...,9f9773533c6111e9cb030dbdb3260582,HT171B1-S1H9A1-4_right__20230512.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,8cab6bfb5753663c397d2ac196971494,HT171B1-S1H9A1-4_right_20230512.csv
HT171B1-S1H9A1-A4_left_20230526,HT171B1,HT171B1-S1H9A1-A4_left,20230526,/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H...,e6fffc8c86acf6f0046cf25274eb2cdb,HT171B1-S1H9A1-A4_left__20230526.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,152c1b2b48a7e0c97de235b6b0d8d981,HT171B1-S1H9A1-A4_left_20230526.csv
HT171B1-S1H9A1-A4_right_20230526,HT171B1,HT171B1-S1H9A1-A4_right,20230526,/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H...,dffdc0264ca6a57f952797f009b0256c,HT171B1-S1H9A1-A4_right__20230526.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,152c1b2b48a7e0c97de235b6b0d8d981,HT171B1-S1H9A1-A4_right_20230526.csv
HT206B1-H1_20220625,HT206B1,HT206B1-H1,20220625,/diskmnt/primary/CODEX/HTAN/20220625-HT206B1-H...,379826a5ffbbc6d03ca34a54015cc8cd,HT206B1-H1__20220625.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,9b0e6fa14075adab187fb199a5743f66,HT206B1-H1_20220625.csv
HT206B1-S1H2L4_20221028,HT206B1,HT206B1-S1H2L4,20221028,/diskmnt/primary/CODEX/HTAN/HT206B1-S1H2L4_LRG...,471df04bd3b1d2c9e6fcf977319ce83a,HT206B1-S1H2L4__20221028.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,170eb95c2b9e63b159f16ed03815de03,HT206B1-S1H2L4_20221028.csv
HT206B1-S1H5_left_20230701,HT206B1,HT206B1-S1H5_left,20230701,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...,32536b3b808f836f4c062d29b952bb25,HT206B1-S1H5_left__20230701.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,bd940309f07f1859404c3de08fc604eb,HT206B1-S1H5_left_20230701.csv
HT206B1-S1H5_right_20230701,HT206B1,HT206B1-S1H5_right,20230701,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...,f7e4b6328b5032475eadb5bcb543e4d9,HT206B1-S1H5_right__20230701.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,bd940309f07f1859404c3de08fc604eb,HT206B1-S1H5_right_20230701.csv


In [99]:
link.to_csv(os.path.join(output_dir, 'matt_mapping_table.csv'), sep=',')

#### matt metadata spreadsheet helpers

In [148]:
fps = """CODEX_Imaging_level_2_BRCA23/_HT271B1-S1H6A5__20230420.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT110B1-S1H4__20221216.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT110B1-S1H4_20221216.csv
CODEX_Imaging_level_2_BRCA23/HT171B1-S1H9A1-4_left__20230512.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT171B1-S1H9A1-4_left_20230512.csv
CODEX_Imaging_level_2_BRCA23/HT171B1-S1H9A1-4_right__20230512.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT171B1-S1H9A1-4_right_20230512.csv
CODEX_Imaging_level_2_BRCA23/HT171B1-S1H9A1-A4_left__20230526.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT171B1-S1H9A1-A4_left_20230526.csv
CODEX_Imaging_level_2_BRCA23/HT171B1-S1H9A1-A4_right__20230526.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT171B1-S1H9A1-A4_right_20230526.csv
CODEX_Imaging_level_2_BRCA23/HT206B1-H1__20220625.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT206B1-H1_20220625.csv
CODEX_Imaging_level_2_BRCA23/HT206B1-S1H2L4__20221028.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT206B1-S1H2L4_20221028.csv
CODEX_Imaging_level_2_BRCA23/HT206B1-S1H5_left__20230701.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT206B1-S1H5_left_20230701.csv
CODEX_Imaging_level_2_BRCA23/HT206B1-S1H5_right__20230701.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT206B1-S1H5_right_20230701.csv
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4__20221216.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4_20221216.csv
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4A4__20230420.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4A4_20230420.csv
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4A4_left__20230512.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4A4_left__20230526.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4A4_left_20230512.csv
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4A4_left_20230526.csv
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4A4_right__20230512.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4A4_right__20230526.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4A4_right_20230512.csv
CODEX_Imaging_level_2_BRCA23/HT243B1-S1H4A4_right_20230526.csv
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_20230420.csv
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_left__20230512.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_left__20230526.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_left_20230512.csv
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_left_20230526.csv
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_middle__20230526.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_middle_20230526.csv
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_right__20230512.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_right__20230526.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_right_20230512.csv
CODEX_Imaging_level_2_BRCA23/HT271B1-S1H6A5_right_20230526.csv
CODEX_Imaging_level_2_BRCA23/HT297B1-S1H1_left__20230701.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT297B1-S1H1_left_20230701.csv
CODEX_Imaging_level_2_BRCA23/HT297B1-S1H1_right__20230701.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT297B1-S1H1_right_20230701.csv
CODEX_Imaging_level_2_BRCA23/HT305B1-S1H1__20221216.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT305B1-S1H1_20221216.csv
CODEX_Imaging_level_2_BRCA23/HT308B1-S1H5A4__20230420.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT308B1-S1H5A4_20230420.csv
CODEX_Imaging_level_2_BRCA23/HT308B1-S1H5A4_left__20230512.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT308B1-S1H5A4_left__20230526.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT308B1-S1H5A4_left_20230512.csv
CODEX_Imaging_level_2_BRCA23/HT308B1-S1H5A4_left_20230526.csv
CODEX_Imaging_level_2_BRCA23/HT308B1-S1H5A4_right__20230512.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT308B1-S1H5A4_right__20230526.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT308B1-S1H5A4_right_20230512.csv
CODEX_Imaging_level_2_BRCA23/HT308B1-S1H5A4_right_20230526.csv
CODEX_Imaging_level_2_BRCA23/HT323B1-A1__20220210.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT323B1-A1_20220210.csv
CODEX_Imaging_level_2_BRCA23/HT323B1-H1__20220804.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT323B1-H1_20220804.csv
CODEX_Imaging_level_2_BRCA23/HT323B1-H1A4__20220210.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT323B1-H1A4_20220210.csv
CODEX_Imaging_level_2_BRCA23/HT323B1-H3__20220211.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT323B1-H3_20220211.csv
CODEX_Imaging_level_2_BRCA23/HT339B1-H1A1__20220302.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT339B1-H1A1_20220302.csv
CODEX_Imaging_level_2_BRCA23/HT339B1-H2A1__20220302.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT339B1-H2A1_20220302.csv
CODEX_Imaging_level_2_BRCA23/HT339B2-H2_left__20230701.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT339B2-H2_left_20230701.csv
CODEX_Imaging_level_2_BRCA23/HT339B2-H2_right__20230701.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT339B2-H2_right_20230701.csv
CODEX_Imaging_level_2_BRCA23/HT365B1_S1H1__20230221.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT365B1_S1H1_20230221.csv
CODEX_Imaging_level_2_BRCA23/HT397B1-H2A2__20220505.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT397B1-H2A2_20220505.csv
CODEX_Imaging_level_2_BRCA23/HT397B1-H3A1__20220429.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT397B1-H3A1_20220429.csv
CODEX_Imaging_level_2_BRCA23/HT397B1-S1H3A1-U2__20230315.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT397B1-S1H3A1-U2_20230315.csv
CODEX_Imaging_level_2_BRCA23/HT397B1-S1H3A1-U22__20230316.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT397B1-S1H3A1-U22__20230413.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT397B1-S1H3A1-U22_20230316.csv
CODEX_Imaging_level_2_BRCA23/HT397B1-S1H3A1-U22_20230413.csv
CODEX_Imaging_level_2_BRCA23/HT397B1-S1H3A1-U31__20230407.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT397B1-S1H3A1-U31_20230407.csv
CODEX_Imaging_level_2_BRCA23/HT480B1-S1H2__20220915.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT480B1-S1H2__20220917.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT480B1-S1H2_20220915.csv
CODEX_Imaging_level_2_BRCA23/HT480B1-S1H2_20220917.csv
CODEX_Imaging_level_2_BRCA23/HT565B1-H2__20230427.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT565B1-H2_20230427.csv
CODEX_Imaging_level_2_BRCA23/HT565B1-S1H2_left__20230701.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT565B1-S1H2_left_20230701.csv
CODEX_Imaging_level_2_BRCA23/HT565B1-S1H2_right__20230701.ome.tiff
CODEX_Imaging_level_2_BRCA23/HT565B1-S1H2_right_20230701.csv""".split('\n')
fps = [fp.split('/')[-1] for fp in fps]
len(fps)

98

In [149]:
meta = pd.read_csv(os.path.join(output_dir, 'file_level_metadata.csv'))
meta

,Component,Filename,File Format,HTAN Participant ID,HTAN Parent Biospecimen ID,HTAN Data File ID,Channel Metadata Filename,Imaging Assay Type,Protocol Link,Workflow Start Datetime,...,PhysicalSizeZ,PhysicalSizeZUnit,Pixels BigEndian,PlaneCount,SizeC,SizeT,SizeX,SizeY,SizeZ,PixelType
0,ImagingLevel2,HT110B1-S1H4__20221216.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT110B1-S1H4__20221216.csv,CODEX,NaN,NaN,...,NaN,NaN,False,26,26,1,16000,15000,1,uint8
1,ImagingLevel2,HT171B1-S1H9A1-4_left__20230512.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT171B1-S1H9A1-4_left__20230512.csv,CODEX,NaN,NaN,...,NaN,NaN,False,33,33,1,13000,9000,1,uint8
2,ImagingLevel2,HT171B1-S1H9A1-4_right__20230512.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT171B1-S1H9A1-4_right__20230512.csv,CODEX,NaN,NaN,...,NaN,NaN,False,33,33,1,14000,9000,1,uint8
3,ImagingLevel2,HT171B1-S1H9A1-A4_left__20230526.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT171B1-S1H9A1-A4_left__20230526.csv,CODEX,NaN,NaN,...,NaN,NaN,False,34,34,1,11000,7000,1,uint8
4,ImagingLevel2,HT171B1-S1H9A1-A4_right__20230526.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT171B1-S1H9A1-A4_right__20230526.csv,CODEX,NaN,NaN,...,NaN,NaN,False,34,34,1,12000,9000,1,uint8
5,ImagingLevel2,HT206B1-H1__20220625.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT206B1-H1__20220625.csv,CODEX,NaN,NaN,...,NaN,NaN,False,29,29,1,14784,20160,1,uint8
6,ImagingLevel2,HT206B1-S1H2L4__20221028.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT206B1-S1H2L4__20221028.csv,CODEX,NaN,NaN,...,NaN,NaN,False,20,20,1,11000,12000,1,uint8
7,ImagingLevel2,HT206B1-S1H5_left__20230701.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT206B1-S1H5_left__20230701.csv,CODEX,NaN,NaN,...,NaN,NaN,False,37,37,1,11000,10000,1,uint8
8,ImagingLevel2,HT206B1-S1H5_right__20230701.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT206B1-S1H5_right__20230701.csv,CODEX,NaN,NaN,...,NaN,NaN,False,37,37,1,9000,10000,1,uint8
9,ImagingLevel2,HT243B1-S1H4A4__20230420.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT243B1-S1H4A4__20230420.csv,CODEX,NaN,NaN,...,NaN,NaN,False,31,31,1,16000,10000,1,uint8


In [150]:
blank = pd.DataFrame(data=np.full((1, meta.shape[1]), None))
meta = pd.concat((meta, blank))
meta


,Component,Filename,File Format,HTAN Participant ID,HTAN Parent Biospecimen ID,HTAN Data File ID,Channel Metadata Filename,Imaging Assay Type,Protocol Link,Workflow Start Datetime,...,37,38,39,40,41,42,43,44,45,46
0,ImagingLevel2,HT110B1-S1H4__20221216.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT110B1-S1H4__20221216.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ImagingLevel2,HT171B1-S1H9A1-4_left__20230512.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT171B1-S1H9A1-4_left__20230512.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ImagingLevel2,HT171B1-S1H9A1-4_right__20230512.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT171B1-S1H9A1-4_right__20230512.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ImagingLevel2,HT171B1-S1H9A1-A4_left__20230526.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT171B1-S1H9A1-A4_left__20230526.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ImagingLevel2,HT171B1-S1H9A1-A4_right__20230526.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT171B1-S1H9A1-A4_right__20230526.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ImagingLevel2,HT206B1-H1__20220625.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT206B1-H1__20220625.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ImagingLevel2,HT206B1-S1H2L4__20221028.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT206B1-S1H2L4__20221028.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ImagingLevel2,HT206B1-S1H5_left__20230701.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT206B1-S1H5_left__20230701.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ImagingLevel2,HT206B1-S1H5_right__20230701.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT206B1-S1H5_right__20230701.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ImagingLevel2,HT243B1-S1H4A4__20230420.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT243B1-S1H4A4__20230420.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
fp_to_idx = {fp:i for i, fp in enumerate(meta['Filename'])}
# fp_to_idx.update({re.sub(r'^(.*_)_([0-9]+.csv)$', r'\1\2', fp):i
#                   for i, fp in enumerate(meta['Channel Metadata Filename'])})


In [152]:
idxs = [fp_to_idx[fp] if fp in fp_to_idx else meta.shape[0] - 1 for fp in fps]
idxs

[15,
 0,
 49,
 1,
 49,
 2,
 49,
 3,
 49,
 4,
 49,
 5,
 49,
 6,
 49,
 7,
 49,
 8,
 49,
 14,
 49,
 9,
 49,
 10,
 11,
 49,
 49,
 12,
 13,
 49,
 49,
 49,
 16,
 17,
 49,
 49,
 18,
 49,
 19,
 20,
 49,
 49,
 21,
 49,
 22,
 49,
 23,
 49,
 24,
 49,
 25,
 26,
 49,
 49,
 27,
 28,
 49,
 49,
 29,
 49,
 31,
 49,
 30,
 49,
 32,
 49,
 33,
 49,
 34,
 49,
 35,
 49,
 36,
 49,
 37,
 49,
 38,
 49,
 39,
 49,
 42,
 49,
 40,
 41,
 49,
 49,
 43,
 49,
 44,
 45,
 49,
 49,
 46,
 49,
 47,
 49,
 48,
 49]

In [153]:
reindexed = meta.iloc[idxs]
reindexed

,Component,Filename,File Format,HTAN Participant ID,HTAN Parent Biospecimen ID,HTAN Data File ID,Channel Metadata Filename,Imaging Assay Type,Protocol Link,Workflow Start Datetime,...,37,38,39,40,41,42,43,44,45,46
15,ImagingLevel2,_HT271B1-S1H6A5__20230420.ome.tiff,OME-TIFF,NaN,NaN,NaN,_HT271B1-S1H6A5__20230420.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ImagingLevel2,HT110B1-S1H4__20221216.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT110B1-S1H4__20221216.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ImagingLevel2,HT171B1-S1H9A1-4_left__20230512.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT171B1-S1H9A1-4_left__20230512.csv,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
47,ImagingLevel2,HT565B1-S1H2_left__20230701.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT565B1-S1H2_left__20230701.csv,CODEX,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
48,ImagingLevel2,HT565B1-S1H2_right__20230701.ome.tiff,OME-TIFF,NaN,NaN,NaN,HT565B1-S1H2_right__20230701.csv,CODEX,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [154]:
link_reindexed = pd.concat((
    link, pd.DataFrame(data=np.full((1, link.shape[1]), None))
))
link_reindexed = link_reindexed.iloc[idxs]
link_reindexed

,case,specimen,date,katmai filepath,md5,Filename,katmai Channel Metadata filepath,md5 channel metadata file,Channel Metadata Filename,0,1,2,3,4,5,6,7,8
HT271B1-S1H6A5_20230420,HT271B1,HT271B1-S1H6A5,20230420.0,/diskmnt/primary/CODEX/HTAN/041923_BRCA_HT171B...,a04754898f10b05758622967bd508201,_HT271B1-S1H6A5__20230420.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,82d12f4d05c4d07314d7ffaf8b21845e,HT271B1-S1H6A5_20230420.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HT110B1-S1H4_20221216,HT110B1,HT110B1-S1H4,20221216.0,/diskmnt/primary/CODEX/HTAN/HT243B1-S1H4_HT110...,b880f7f6a29ae3baeca397abd47a9dd8,HT110B1-S1H4__20221216.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,10247040a2ab142c6ce1910225a29022,HT110B1-S1H4_20221216.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HT171B1-S1H9A1-4_left_20230512,HT171B1,HT171B1-S1H9A1-4_left,20230512.0,/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...,8cba6e147fcffb112a0490d138fe1db5,HT171B1-S1H9A1-4_left__20230512.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,8cab6bfb5753663c397d2ac196971494,HT171B1-S1H9A1-4_left_20230512.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None
HT565B1-S1H2_left_20230701,HT565B1,HT565B1-S1H2_left,20230701.0,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...,4e419572f3244a404e908436f33a013e,HT565B1-S1H2_left__20230701.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,bd940309f07f1859404c3de08fc604eb,HT565B1-S1H2_left_20230701.csv,None,None,None,None,None,None,None,None,None
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None
HT565B1-S1H2_right_20230701,HT565B1,HT565B1-S1H2_right,20230701.0,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...,4a851cb987e9addee6daf37954d9b7de,HT565B1-S1H2_right__20230701.ome.tiff,/diskmnt/Projects/Users/estorrs/imaging-analys...,bd940309f07f1859404c3de08fc604eb,HT565B1-S1H2_right_20230701.csv,None,None,None,None,None,None,None,None,None


In [155]:
p = 'CODEX_Imaging_level_2_BRCA23'
reindexed['Channel Metadata Filename'] = [os.path.join(p, x) if not pd.isnull(x) else x
                                          for x in reindexed['Channel Metadata Filename']]
reindexed

/diskmnt/Projects/Users/estorrs/sandbox/tmp/ipykernel_192165/1568631790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reindexed['Channel Metadata Filename'] = [os.path.join(p, x) if not pd.isnull(x) else x


,Component,Filename,File Format,HTAN Participant ID,HTAN Parent Biospecimen ID,HTAN Data File ID,Channel Metadata Filename,Imaging Assay Type,Protocol Link,Workflow Start Datetime,...,37,38,39,40,41,42,43,44,45,46
15,ImagingLevel2,_HT271B1-S1H6A5__20230420.ome.tiff,OME-TIFF,NaN,NaN,NaN,CODEX_Imaging_level_2_BRCA23/_HT271B1-S1H6A5__...,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ImagingLevel2,HT110B1-S1H4__20221216.ome.tiff,OME-TIFF,NaN,NaN,NaN,CODEX_Imaging_level_2_BRCA23/HT110B1-S1H4__202...,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ImagingLevel2,HT171B1-S1H9A1-4_left__20230512.ome.tiff,OME-TIFF,NaN,NaN,NaN,CODEX_Imaging_level_2_BRCA23/HT171B1-S1H9A1-4_...,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
47,ImagingLevel2,HT565B1-S1H2_left__20230701.ome.tiff,OME-TIFF,NaN,NaN,NaN,CODEX_Imaging_level_2_BRCA23/HT565B1-S1H2_left...,CODEX,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
48,ImagingLevel2,HT565B1-S1H2_right__20230701.ome.tiff,OME-TIFF,NaN,NaN,NaN,CODEX_Imaging_level_2_BRCA23/HT565B1-S1H2_righ...,CODEX,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [156]:
reindexed['WUSTL Path'] = link_reindexed.loc[:, 'katmai filepath'].to_list()
reindexed

/diskmnt/Projects/Users/estorrs/sandbox/tmp/ipykernel_192165/3569402406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reindexed['WUSTL Path'] = link_reindexed.loc[:, 'katmai filepath'].to_list()


,Component,Filename,File Format,HTAN Participant ID,HTAN Parent Biospecimen ID,HTAN Data File ID,Channel Metadata Filename,Imaging Assay Type,Protocol Link,Workflow Start Datetime,...,38,39,40,41,42,43,44,45,46,WUSTL Path
15,ImagingLevel2,_HT271B1-S1H6A5__20230420.ome.tiff,OME-TIFF,NaN,NaN,NaN,CODEX_Imaging_level_2_BRCA23/_HT271B1-S1H6A5__...,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/diskmnt/primary/CODEX/HTAN/041923_BRCA_HT171B...
0,ImagingLevel2,HT110B1-S1H4__20221216.ome.tiff,OME-TIFF,NaN,NaN,NaN,CODEX_Imaging_level_2_BRCA23/HT110B1-S1H4__202...,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/diskmnt/primary/CODEX/HTAN/HT243B1-S1H4_HT110...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,NaN
1,ImagingLevel2,HT171B1-S1H9A1-4_left__20230512.ome.tiff,OME-TIFF,NaN,NaN,NaN,CODEX_Imaging_level_2_BRCA23/HT171B1-S1H9A1-4_...,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,NaN
47,ImagingLevel2,HT565B1-S1H2_left__20230701.ome.tiff,OME-TIFF,NaN,NaN,NaN,CODEX_Imaging_level_2_BRCA23/HT565B1-S1H2_left...,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,NaN
48,ImagingLevel2,HT565B1-S1H2_right__20230701.ome.tiff,OME-TIFF,NaN,NaN,NaN,CODEX_Imaging_level_2_BRCA23/HT565B1-S1H2_righ...,CODEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT20...


In [157]:
reindexed.to_csv(os.path.join(output_dir, 'matt_metadata.csv'), sep=',', index=False, encoding='UTF-8')

In [158]:
os.path.join(output_dir, 'matt_metadata.csv')

'/diskmnt/Projects/Users/estorrs/imaging-analysis/data/brca_subtype/htan_upload/matt_metadata.csv'

In [91]:
## check for image existence
for fp in link['katmai filepath']:
    assert os.path.exists(fp), f'{fp}'

In [92]:
for fp in link['katmai Channel Metadata filepath']:
    assert os.path.exists(fp), f'{fp}'